# I. Introduction

The software implements ‘Automatic Differentiation’ (AD). This is a technique to computationally evaluate the derivative of a specified function. Importantly, AD is not the same as symbolic differentiation or numerical differentiation, and holds important advantages over both. Symbolic differentiation, which is equivalent to analytically solving differential equations, can find the exact solution (with machine precision), but is very computationally expensive, and so with very large functions can be infeasible. Numerical differentiation, which uses the finite-difference approximation method, is computationally efficient, but is ultimately only approximate, and can be subject to both rounding error and discretisation error, meaning that it cannot be perfectly accurate. Both of these ‘traditional’ methods of differentiation run into problems and inefficiencies when calculating higher derivatives and partial derivatives with respect to many inputs (which is an important component of gradient-based optimisation methods). 
Automatic differentiation solves all these problems as it is able to solve derivatives to machine precision with comparative computational efficiency. As a result, automatic differentiation has incredibly important applications; in its ‘reverse-mode’ (discussed below), it is the basis of back-propagation, a fundamental process in neural network machine learning algorithms - as such this technique is leveraged by open-source machine learning libraries such as TensorFlow. A result of its efficient accuracy and iterative method, AD is capable of algorithmic differentiation: Because of the fact that every computer program, from mathematical algorithms to web-pages, can be expressed as a sequence and combination of arithmetic operations and elementary functions, the derivative of any computer program can be found using automatic differentiation.

# II. Background

Automatic differentiation is essentially the iterative application of the chain-rule. As mentioned above, any function can be considered a sequence of basic arithmetic operations or elementary functions (+,-,x,/,sin,cos,exp etc.) and so any function can be interpreted in the following way (albeit often less simply):
	$$y = f(x) = f(g(h((x)))$$

This can be rewritten as:
$$y = f(g(h(x0) = f(g(x1) = f(x2)) = x3$$
	
Often, this decomposition is represented as an acyclic, directed computational graph that illustrates the route from the base function x0 to y.
In forward mode, automatic differentiation works by decomposing the function into this structure, and working through each component function finding the derivative using the chain rule ‘inside out’. That is to say, dx0/dx is found first, following by dx1/dx and so on until dy/dx itself is found. All this requires initial values to be set for x0, and x0’.
	Reverse mode, however, works in the opposite direction; rather than finding the derivative of the most fundamental component, and then finding the derivative of parent expressions in terms of these children components recursively until the final gradient is found, reverse mode goes the other way. It finds the derivative of each ‘child’ function in terms of its parent function recursively until the basic level derivative is found, at which point the final gradient can be found.
	One way of achieving forward mode AD is to use dual numbers. These are an extension of real numbers, somewhat analogous to imaginary numbers, such that every number additionally contains a dual component, $\epsilon$, where $\epsilon^2$ = 0. Given any polynomial function (or, in fact, any analytic real function via its Taylor series), if we replace x with (x+x'$\epsilon$), we find that the function will become: f(x) + f'(x)$\epsilon$. This provides a routine to automatically compute the derivative of the function f(x), and so is used in forward AD.

Sources: https://en.wikipedia.org/wiki/Automatic_differentiation
	   http://www.columbia.edu/~ahd2125/post/2015/12/5/


# III. How to Use

The current AutoDiff package implements forward-mode automatic differentiation. Users should import the package using the following commands:

```python
>>> import AutoDiff 
>>> #or
>>> from AutoDiff import AD, AD_create, AD_stack
```

Importation of individual methods is not necessary but possible. While the current package employs instances and methods of Numpy and Math, importation of these modules is unnecessary.  

To instantiate an automatic differentiation [AD] object, users should define variables and values before defining function(s) for differentiation. There is no limit on the number of variables a function can include for AD. By wrapping a variable in AD_create( ), users will have instantiated an AD object: 

```python
>>> #example
>>> a = 5.0
>>> x = AD(a) #returns a as an AD object stored as x
>>> print(x)
    <__main__.AD object at 0x103bi28d7> 
>>> print(x.val)
    5.0
```

Alternatively, users may initiate multiple AD objects by passing an array/list of variables. For example:

```python
>>> x, y, z = 1.0, 3.0, 4.0 #assign values to multiple variables
>>> vars = [x, y, z] #initiate a list of variables
>>> varsAD = AD_create(vars)
>>> print(varsAD)
    [<__main__.AD object at 0x103bi28d7>, <__main__.AD object at 0x124bi28c5>,<__main__.AD object at 3x156bw22c0>]
>>> print(varsAD[1].val)
    3.0
```

Users should then define function(s) for AD, function value and derivative will be returned by calling ".val" and ".der" respectively. 

```python
>>> #example
>>> x, y, z = 1.0, 3.0, 4.0
>>> vars = [x, y, z]
>>> varsAD = AD_create(vars)
>>> function = 2x + 5y**2 + z
>>> print(function.val, function.der)
    53.0, [2.0, 30.0, 1.0] #return partial derivatives in the order of df/dx, df/dy, df/dz
```

In the case where output function is multidimentional, users can define functions and compute values and derivatives as such:

```python
>>> #example
>>> x, y, z = 1.0, 3.0, 4.0
>>> vars = [x, y, z]
>>> varsAD = AD_create(vars)
>>> f1 = x + 2y + 3z
>>> f2 = 2x + y + z
>>> f = AD_stack([f1, f2]) #return f as an AD object containing two AD objects
>>> print(f.val, f.der)
>>> [19.0, 9.0] [[1, 2, 3], [2, 1, 1]]
```

# IV. Software Organization

* Directory Structure 

The following is our envisioned directory structure:

```
CS207-FinalProject\
                   AutoDiff\
                            AutoDiff/
                                    __init__.py
                                    AD.py
                            Tests/
                                 __init__.py
                                 AD_Test.py
                                 setup.cfg
                                 .travis.yml  
                            README.md
                            setup.py
                            LICENSE
```

Speaking from the current state of project, the AutoDiff package contains one module "AD.py," which computes and outputs function value, derivative, and partial derivatives using forward-mode automatic differentiation. This will be the module that users import for automatic differentiation. 

The Tests subdirectory currently contains one module. It includes combinations of tests following pytest to ensure that methods from the AD module are executed correctly, and that proper errors are returned when user inputs are incorrect.   

We plan to implement reverse-mode automatic differentiation upon completing the development of forward-mode automatic differentiation. The implementation will fall under a separate module, "AD_r.py," which will be added to the AutoDiff package. A separate test module, "AD_r_Test.py" will be added to the Tests subdirectory.   

* Test Suite and Package Distribution

We will use Travis CI and Coveralls to host our test suite. 
Travis CI: https://travis-ci.org/CS207-Project-Group-9/cs207-FinalProject.svg?branch=master
Coveralls: https://coveralls.io/repos/github/CS207-Project-Group-9/cs207-FinalProject/badge.svg?branch=master

We will deploy our package using PyPI. 


# V. Implementation

## Class 1: AD

Describes variables in 1-Dimension

#### Attributes
- val: float
    - A numeric value, indicating the value of the current AD instance.
    
- der: array of float
    - Suppose there are m elementary variables, 'der' would be an array of m elements, each representing the derivative value with respect to a certain elementary variable.
    - We store these values in an array so that when our later computation is easier.

#### Methods
0. \__init__:
    - arguments: 
        - val: float, representing the value
        - der: a list of floats, representing the derivatives with respect to elementary variables; optional, default=1
    - sets self.val=val, self.der=der


1. \__add__ & \__radd__: 
    - arguments:
        - self
        - other: a float, int, or AD
    - returns: 
        - if other is an AD -> a new AD instance with new.val = self.val + other.val, new.der = self.der + other.der
        - if other is a numeric value -> a new AD instance with new.val = self.val + other, new.der = self.der


2. \__sub__ & \__rsub__
    - arguments:
		- self
		- other: a float, int, or AD
	- returns: 
		- if other is an AD -> a new AD instance with new.val = self.val - other.val, new.der = self.der - other.der
		- if other is a numeric value -> a new AD instance with new.val = self.val - other, new.der = self.der


3. \__mul__ & \__rmul__
	- arguments:
		- self
		- other: a float, int, or AD
	- returns: 
		- if other is an AD -> a new AD instance with new.val = self.val \* other.val, new.der = self.val \* other.der + self.der \* other.val
		- if other is a numeric value -> a new AD instance with new.val = self.val \* other, new.der = self.der \* other


4. \__div__ & \__rdiv__
	- arguments:
		- self
		- other: a float, int, or AD
	- returns: 
		- if other is an AD -> a new AD instance with new.val = self.val / other.val, new.der = (self.der \* other.val + self.val \* other.der)/(other.val\*\*2)
		- if other is a numeric value -> a new AD instance with new.val = self.val / other, new.der = self.der / other
	- raises:
		- ZeroDivisionError when other.val = 0


5. \__pow__
	- arguments:
		- self
		- k: a float, int, or AD
	- returns:
		- if other is an AD -> a new AD instance with new.val = self.val \*\* other.val, new.der = other.val \* self.val \*\* (other.val - 1) \* self.der + log(self.val) \* self.val \*\* other.val \* other.der
		- if other is a numeric value -> a new AD instance with new.val = self.val \*\* k, new.der = k \* (self.val\*\*(k-1)) \* self.der


6. \__abs__
	- arguments:
		- self
	- returns:
		- a new AD instance with new.val = abs(self.val), new.der = sign(self.val) \* self.der


7. sin
	- arguments:
		- self
	- returns:
		- a new AD instance with new.val = sin(self.val), new.der = cos(self.val) \* self.der


8. cos
	- arguments:
		- self
	- returns:
		- a new AD instance with new.val = cos(self.val), new.der = -sin(self.val) \* self.der


9. tan
	- arguments:
		- self
	- returns:
		- a new AD instance with new.val = cos(self.val), new.der = 1/(cos(self.val) \*\* 2) \* self.der


10. exp
	- arguments:
		- self
	- returns:
		- a new AD instance with new.val = exp(self.val), new.der = exp(self.val) \* self.der


11. log
	- arguments:
		- self
	- returns:
		- a new AD instance with new.val = log(self.val), new.der = self.der/self.val
	- raises: (????)
		exception when self.val <= 0 

12. get_val()
    - arguments:
        - self
    - returns:
        - self.val

13. get_der(i=1)
    - arguments:
        - self
    - returns:
        - self.der

14. \__eq__
    - arguments:
        - self
        - other: an AD instance
    - returns:
        - 'True' if self.val==other.val and self.der==other.der, 'False' otherwise
        
15. \__str__
    - arguments:
        - self
    - returns:
        - a string describing the value and derivatives of the current instance

## Class 2: AD_System

Describes variables in n-Dimension

#### Attributes
- val: array of float
    - Numeric values, indicating the value of each entry in the current AD_System instance.
    
- der: 2D array of float
    - Suppose there are m elementary variables and n variables in the AD_System, 'der' would be a n\*m array of elements, with the (i,j) entry representing the derivative value of the i-th AD with respect to the j-th elementary variable.

#### Methods
0. \__init__:
    - arguments: 
        - a list/array of AD instances
    - sets self.val as a list of 'val' attributes of the input AD instances
    - combines the 'der' attributes of the input AD instances as a 2D array and save as self.der

Class AD_System will also implement the following functions, very similar to the simple AD class:
1. \__add__ & \__radd__
2. \__sub__ & \__rsub__
3. \__mul__ & \__rmul__
4. \__div__ & \__rdiv__
5. \__pow__
6. \__abs__
7. sin
8. cos
9. tan
10. exp
11. log
12. get_val()
13. get_der(i=1)
14. \__eq__
15. \__str__

## External Dependencies

We will need **numpy** for impletation and **pytest** and **doctest** for testing.